In [6]:
import json
import numpy as np

avg_brier = []
avg_acc = []
# with open(f"halawi_output/halawi_v15_metaculus_o1-preview_breadth7_run0.json") as file:
with open("public_output/test_metaculus_full_v17_o1-preview_chunk3_run0.json") as file:
    data= json.load(file)

cors_crowd = []
cors_model = []
avg_scores = []
crowd_scores = []
for market in data:
    # if market['data_source'] != 'metaculus': 
    #     continue
    # if market['question_type'].lower() != 'binary':
    #     continue

    pred = market['predictions'][0]
    # if "2023" in pred:
    #     print(market['response'][0])
    if pred == None:
        continue
        # pred = 0.0
    # print(pred)
    if pred > 2000:
        continue
    if pred > 1:
        # print(pred)
        pred = pred / 100
    # if pred == None:
    #     continue
    crow_pred = market['crowd_pred'][3][-1]
    resolution = market['resolution']

    score = (pred - resolution) ** 2
    crowd_score = (crow_pred - resolution) ** 2

    cors_model.append(int(pred > 0.50) == resolution)
    cors_crowd.append(int(crow_pred> 0.50) == resolution)
    avg_scores.append(score)
    crowd_scores.append(crowd_score)

avg_brier.append(np.mean(avg_scores))
avg_acc.append(np.mean(cors_model))
print(np.mean(avg_scores), np.mean(cors_model), np.mean(cors_crowd), len(cors_model))

print("model")
print(f"acc: {np.mean(cors_model) * 100:.2f}%")
print(f"brier: {np.mean(avg_scores):.4f} / ", len(avg_scores))

print("crowd")
print(f"acc: {np.mean(cors_crowd) * 100:.2f}%")
print(f"brier: {np.mean(crowd_scores):.4f}")

0.11362233986928105 0.8235294117647058 0.8954248366013072 153
model
acc: 82.35%
brier: 0.1136 /  153
crowd
acc: 89.54%
brier: 0.0838


In [10]:
import json
import numpy as np

# with open("halawi_output/halawi_v15_metaculus_grok-2-latest_breadth7_run1_full.json") as file:
with open("public_output/test_metaculus_full_v17_o1-preview_chunk3_run0_full.json") as file:
    data = json.load(file)

cnt = 0
hard_questions = []
avg_scores = []

chunk = 0
cors_model = []
cors_crowd = []
for market in data:
    # if market['data_source'] != 'metaculus': 
    #     continue
    
    pred = market['predictions'][0]
    if pred == None:
        continue
    crow_pred = market['crowd_pred'][3][-1]
    origin_crowd_pred = market['crowd_pred'][0][-1]
    resolution = market['resolution']
    

    model_score = (pred - resolution) ** 2

    crowd_score = (crow_pred - resolution) ** 2
    og_crowd_score = (origin_crowd_pred - resolution) ** 2
    cor_model = int(pred > .50) == resolution
    cor_crowd = int(crow_pred >.50) == resolution
    cors_model.append(cor_model)
    cors_crowd.append(cor_crowd)
    avg_scores.append(model_score)
    if len(market['sources'][0]) == 0:
        cnt += 1
    # if model_score - crowd_score > 0.1:
    if not cor_model and cor_crowd:
        question = market['question']
        hard_questions.append(market)
        print("question:", question)
        print("model: ", pred, " | score:", model_score)
        print("crowd: ", crow_pred, " | score:", crowd_score)
        print("res=", resolution)
        print("=====")

np.mean(cors_crowd), np.mean(cors_model)

question: Will a debate be held between Joe Biden and Donald Trump before the 2024 US presidential election?
model:  0.4  | score: 0.36
crowd:  0.57  | score: 0.18490000000000004
res= 1.0
=====
question: Will salvage operations commence on the cargo vessel "M/V Rubymar" prior to April 1, 2024?
model:  0.65  | score: 0.42250000000000004
crowd:  0.496  | score: 0.24601599999999998
res= 0.0
=====
question: Will UCLA hold its in-person, university-wide graduation commencement ceremonies on June 14, 2024?
model:  0.45  | score: 0.30250000000000005
crowd:  0.628  | score: 0.138384
res= 1.0
=====
question: Will OpenAI investors file lawsuits against the OpenAI board related to the firing of Sam Altman before March 1, 2024?
model:  0.6  | score: 0.36
crowd:  0.215  | score: 0.046224999999999995
res= 0.0
=====
question: Will Leinier Domínguez Pérez be one of the 8 players to participate in the 2024 FIDE Candidates tournament?
model:  0.6  | score: 0.36
crowd:  0.056  | score: 0.0031360000000000

(0.8954248366013072, 0.8235294117647058)

In [22]:
import json
import numpy as np

avg_brier = []
avg_acc = []

file_name = "halawi_v15_metaculus_0_full.json"
file_name = "halawi_v15_metaculus_grok-2-latest_breadth10_run0.json"


# file_name = "halawi_v15_metaculus_breadth7.json"
# file_name = "halawi_v15_0.json"
with open(f"halawi_output/{file_name}") as file:
    data= json.load(file)

all_cors_model = []
all_cors_crowd = []
for chunk in range(0,5):
    cors_crowd = []
    cors_model = []
    avg_scores = []
    crowd_scores = []
    for market in data:
        # if market['data_source'] != 'metaculus': 
        #     continue
        # if market['question_type'].lower() != 'binary':
        #     continue
        
        # print("market['predictions']=",market['predictions'])
        pred = market['predictions'][chunk]
        # assert pred
        crow_pred = market['crowd_pred'][chunk][-1]
        resolution = market['resolution']

        score = (pred - resolution) ** 2
        crowd_score = (crow_pred - resolution) ** 2

        cors_model.append(int(pred > 0.50) == resolution)
        cors_crowd.append(int(crow_pred> 0.50) == resolution)
        avg_scores.append(score)
        crowd_scores.append(crowd_score)

    avg_brier.append(np.mean(avg_scores))
    avg_acc.append(np.mean(cors_model))
    # print(np.mean(avg_scores), np.mean(cors_model), np.mean(cors_crowd), len(cors_model))

    all_cors_model.append(f"{np.mean(cors_model) * 100:.2f}%")
    all_cors_crowd.append(f"{np.mean(cors_crowd) * 100:.2f}%")
    # print("model")
    # print(f"acc: {np.mean(cors_model) * 100:.2f}%")
    # print(f"brier: {np.mean(avg_scores):.4f} / ", len(avg_scores))

    # print("crowd")
    # print(f"acc: {np.mean(cors_crowd) * 100:.2f}%")
    # print(f"brier: {np.mean(crowd_scores):.4f}")

print("all_cors_model=",all_cors_model)
print("all_cors_crowd=",all_cors_crowd)

all_cors_model= ['62.86%', '74.29%', '82.86%', '77.14%', '80.00%']
all_cors_crowd= ['77.14%', '71.43%', '77.14%', '82.86%', '88.57%']


In [2]:
from datetime import datetime

def calculate_prediction_durations(data):
    durations = []

    for question in data:
        crowd_pred = question['crowd_pred']
        first_prediction = datetime.strptime(crowd_pred[0][1], "%Y-%m-%d")
        last_prediction = datetime.strptime(crowd_pred[-1][1], "%Y-%m-%d")
        duration = (last_prediction - first_prediction).days
        durations.append(duration)

    average_duration = sum(durations) / len(durations)
    return average_duration

In [18]:
import json
import numpy as np
from statistics import mean, stdev

def process_file(file_name):
    with open(f"halawi_output/{file_name}") as file:
        data = json.load(file)
    
    all_cors_model = []
    all_cors_crowd = []
    all_scores_model = []
    all_scores_crowd = []
    for chunk in range(0, 5):
        cors_model = []
        cors_crowd = []

        scores_model = []
        scores_crowd = []        
        for market in data:
            question = market['question']
            # if question_to_num_forecast[question] < 20:
            #     print("question_to_num_forecast=",question_to_num_forecast[question])
            #     continue
            
            pred = market['predictions'][chunk]
            if pred == None:
                continue
            if pred > 100:
                pred = -1
            if pred > 1:
                pred = pred / 100
            crow_pred = market['crowd_pred'][chunk][-1]
            resolution = market['resolution']
            
            cors_model.append(int(pred > 0.50) == resolution)
            cors_crowd.append(int(crow_pred > 0.50) == resolution)

            scores_model.append((pred - resolution) ** 2)
            scores_crowd.append((crow_pred - resolution) ** 2)
        
        total_date = calculate_prediction_durations(data)

        all_cors_model.append(np.mean(cors_model) * 100)
        all_cors_crowd.append(np.mean(cors_crowd) * 100)
        all_scores_model.append(np.mean(scores_model))
        all_scores_crowd.append(np.mean(scores_crowd))
    
    return all_cors_model, all_cors_crowd, all_scores_model, all_scores_crowd, total_date

# Process all files
results = []
for run in range(5):
# for run in (0,0):
    # file_name = f"halawi_v15_metaculus_full_breadth7_run{run}.json"
    # file_name = f"halawi_v15_metaculus_grok-2-latest_breadth7_run{run}.json"
    file_name = f"halawi_v15_metaculus_o1-preview_breadth7_run{run}.json"
    # file_name = f"halawi_v16_metaculus_gpt-4o-2024-05-13_breadth7_run{run}.json"
    results.append(process_file(file_name))

# Separate model and crowd results
model_results = [result[0] for result in results]
model_scores = [result[2] for result in results]

crowd_results = [result[1] for result in results]
crowd_scores = [result[3] for result in results]
total_date = [result[-1] for result in results]

# Calculate averages and standard deviations
model_averages = [mean(chunk) for chunk in zip(*model_results)]
model_stdevs = [stdev(chunk) for chunk in zip(*model_results)]
model_scores_averages = [mean(chunk) for chunk in zip(*model_scores)]
model_scores_std =  [stdev(chunk) for chunk in zip(*model_scores)]


crowd_averages = [mean(chunk) for chunk in zip(*crowd_results)]
crowd_stdevs = [stdev(chunk) for chunk in zip(*crowd_results)]
crowd_scores_averages = [mean(chunk) for chunk in zip(*crowd_scores)]
crowd_scores_std = [stdev(chunk) for chunk in zip(*crowd_scores)]


# Print results
print("Model Results:")
for i, (avg, std) in enumerate(zip(model_averages, model_stdevs)):
    print(f"Chunk {i}:Acc {avg:.2f}% ± {std:.2f}% | Brier: {model_scores_averages[i]:.4f} ± {model_scores_std[i]:.4f}")

model_stdevs_squared = [std**2 for std in model_stdevs[:-1]]
model_overall_std = np.sqrt(np.mean(model_stdevs_squared))
print("avg:", np.mean(model_averages), "± ", np.mean(model_overall_std))

print("\nCrowd Results:")
for i, (avg, std) in enumerate(zip(crowd_averages, crowd_stdevs)):
    print(f"Chunk {i}: {avg:.2f}% ± {std:.2f}% | Brier: {crowd_scores_averages[i]:.4f} ± {crowd_scores_std[i]:.4f}")

print("avg:", np.mean(crowd_averages))

Model Results:
Chunk 0:Acc 73.71% ± 4.24% | Brier: 0.1736 ± 0.0163
Chunk 1:Acc 80.00% ± 5.35% | Brier: 0.1746 ± 0.0660
Chunk 2:Acc 88.47% ± 5.28% | Brier: 0.0936 ± 0.0350
Chunk 3:Acc 86.29% ± 2.39% | Brier: 0.1031 ± 0.0155
Chunk 4:Acc 90.29% ± 4.78% | Brier: 0.0826 ± 0.0310
avg: 83.75064935064935 ±  4.475717358837765

Crowd Results:
Chunk 0: 77.14% ± 0.00% | Brier: 0.1704 ± 0.0000
Chunk 1: 71.43% ± 0.00% | Brier: 0.1375 ± 0.0000
Chunk 2: 77.47% ± 0.74% | Brier: 0.1196 ± 0.0057
Chunk 3: 82.86% ± 0.00% | Brier: 0.1062 ± 0.0000
Chunk 4: 88.57% ± 0.00% | Brier: 0.0723 ± 0.0000
avg: 79.4943722943723


In [76]:
import json

with open("/data/long_phan/forecasting_platform/backend/src/benchmarks/data/metaculus_sample_full.json") as file:
    data = json.load(file)

durations = []

for question in data:
    crowd_pred = question['crowd_pred']
    first_prediction = datetime.strptime(crowd_pred[0][1], "%Y-%m-%d")
    last_prediction = datetime.strptime(crowd_pred[-1][1], "%Y-%m-%d")
    
    third_time = datetime.strptime(crowd_pred[3][1], "%Y-%m-%d")
    cut_off = datetime.strptime("2023-12-01", "%Y-%m-%d")
    if third_time < cut_off:
        print(third_time, cut_off)
        print(question['question'])
        print("===")

    duration = (last_prediction - first_prediction).days
    # if duration < 7:
    #     print(question['question'])
    #     continue
    durations.append(duration)

average_duration = mean(durations)
average_duration, min(durations), max(durations)

2023-11-18 00:00:00 2023-12-01 00:00:00
Will Erling Haaland score the most goals in the 2023/24 Premier League season?
===
2023-11-30 00:00:00 2023-12-01 00:00:00
Will Ursula von der Leyen be re-appointed as President of the European Commission following the 2024 European elections?
===
2023-11-12 00:00:00 2023-12-01 00:00:00
Will the UK and India have signed an FTA before the next UK General Election?
===
2023-10-24 00:00:00 2023-12-01 00:00:00
Will ChatGPT be available in Europe on June 30, 2024?
===
2023-11-12 00:00:00 2023-12-01 00:00:00
Will the right-wing incumbent BJP win the 2024 national election in India?
===


(106.51412429378531, 0, 310)

In [86]:
from datasets import load_dataset, concatenate_datasets
from datetime import datetime
import json
import requests
from tqdm import tqdm

with open("/data/long_phan/forecasting_platform/backend/src/benchmarks/data/halawi_dataset_metaculus_full.json") as file:
    anchor_data = {}
    for d in json.load(file):
        anchor_data[d['question']] = d

# Load both test and validation splits
dataset = load_dataset("YuehHanChen/forecasting", split="test")
question_to_num_forecast = {}
for row in tqdm(dataset):
    question = row['question']
    if question in anchor_data:
        try:
            response = requests.get(row['url']).json()
            number_of_forecasters = response['number_of_forecasters']
            print("number_of_forecasters=",number_of_forecasters)
            question_to_num_forecast[question] = number_of_forecasters
        except:
            continue

 26%|██▋       | 241/914 [00:00<00:01, 671.18it/s]

number_of_forecasters= 100
number_of_forecasters= 112
number_of_forecasters= 83
number_of_forecasters= 39
number_of_forecasters= 45
number_of_forecasters= 18
number_of_forecasters= 178
number_of_forecasters= 61
number_of_forecasters= 58
number_of_forecasters= 70
number_of_forecasters= 53
number_of_forecasters= 56
number_of_forecasters= 53
number_of_forecasters= 50
number_of_forecasters= 92
number_of_forecasters= 8
number_of_forecasters= 58
number_of_forecasters= 32
number_of_forecasters= 32
number_of_forecasters= 32
number_of_forecasters= 91
number_of_forecasters= 73
number_of_forecasters= 100
number_of_forecasters= 47
number_of_forecasters= 62
number_of_forecasters= 38
number_of_forecasters= 9
number_of_forecasters= 27
number_of_forecasters= 65
number_of_forecasters= 80
number_of_forecasters= 58
number_of_forecasters= 9
number_of_forecasters= 59
number_of_forecasters= 86


100%|██████████| 914/914 [00:11<00:00, 79.68it/s] 

number_of_forecasters= 43


In [83]:
len(question_to_num_forecast)

35

In [77]:
import json

with open("/data/long_phan/forecasting_platform/backend/src/benchmarks/data/halawi_dataset_polymarket.json") as file:
    data = json.load(file)

durations = []

for question in data:
    crowd_pred = question['crowd_pred']
    first_prediction = datetime.strptime(crowd_pred[0][1], "%Y-%m-%d")
    last_prediction = datetime.strptime(crowd_pred[-1][1], "%Y-%m-%d")
    
    third_time = datetime.strptime(crowd_pred[3][1], "%Y-%m-%d")
    cut_off = datetime.strptime("2023-12-01", "%Y-%m-%d")
    # if third_time < cut_off:
    #     print(third_time, cut_off)
    #     print(question['question'])
    #     print("===")

    duration = (last_prediction - first_prediction).days
    # if duration < 7:
    #     print(question['question'])
    #     continue
    durations.append(duration)

average_duration = mean(durations)
average_duration, min(durations), max(durations)

(9.924242424242424, 4, 28)

In [ ]:
Model Results:
Chunk 0: 68.57% ± 0.00%
Chunk 1: 69.52% ± 1.65%
Chunk 2: 75.24% ± 1.65%
Chunk 3: 82.86% ± 4.95%
Chunk 4: 80.95% ± 3.30%

Crowd Results:
Chunk 0: 77.14% ± 0.00%
Chunk 1: 71.43% ± 0.00%
Chunk 2: 77.14% ± 0.00%
Chunk 3: 82.86% ± 0.00%
Chunk 4: 88.57% ± 0.00%